<a href="https://colab.research.google.com/github/sadhiin/Attention-Is-All-You-Need/blob/main/Transformer_Trainer_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Remove non alphanumeric characters for simple training

In [1]:
!pip install --upgrade torch -q

In [2]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN

    def batch_tokenize(self, batch, start_token, end_token):

        def tokenize(sentence, start_token, end_token):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())

    def forward(self, x, start_token, end_token): # sentence
        x = self.batch_tokenize(x, start_token, end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        return x


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out


class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x


class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_attention_mask):
        residual_x = x.clone()
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x.clone()
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x

class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x

class Encoder(nn.Module):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask)
        return x


class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, y, mask):
        batch_size, sequence_length, d_model = x.size() # in practice, this is the same for both languages...so we can technically combine with normal attention
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask) # We don't need the mask for cross attention, removing in outer function!
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out


class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone()
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y.clone()
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y.clone()
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y


class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

class Decoder(nn.Module):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y


class Transformer(nn.Module):
    def __init__(self,
                d_model,
                ffn_hidden,
                num_heads,
                drop_prob,
                num_layers,
                max_sequence_length,
                bn_vocab_size,
                english_to_index,
                kannada_to_index,
                START_TOKEN,
                END_TOKEN,
                PADDING_TOKEN
                ):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, kannada_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, bn_vocab_size)
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    def forward(self,
                x,
                y,
                encoder_self_attention_mask=None,
                decoder_self_attention_mask=None,
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False, # We should make this true
                dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out

In [3]:
# from transformer import Transformer # A local file transformer.py
import torch
import numpy as npp
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [4]:
# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

bangla_vocabulary = [
    START_TOKEN, ' ', '!', '"', '#', '€', '$', '%', '&', "'", '(', ')', '*', '+',
    ',', ';', '-', '.', '/', ':', '<', '=', '>', '?', '@',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯'
    'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ',
    'ক', 'খ', 'গ', 'ঘ', 'ঙ',
    'চ', 'ছ', 'জ', 'ঝ', 'ঞ',
    'ট', 'ঠ', 'ড', 'ঢ', 'ণ',
    'ত', 'থ', 'দ', 'ধ', 'ন',
    'প', 'ফ', 'ব', 'ভ', 'ম',
    'য', 'র', 'ল', 'শ', 'ষ',
    'স', 'হ', 'ড়', 'ঢ', 'য়', 'ঃ', 'ং', '্',
    'ো', 'ি', 'ৌ', 'ৗ', 'ৎ', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', '।',
    'ঁ', 'া', '্র', '্ন',
    PADDING_TOKEN, END_TOKEN
]
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [5]:
index_to_bangla = {k:v for k,v in enumerate(bangla_vocabulary)}
bangla_to_index = {v:k for k,v in enumerate(bangla_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [6]:
bn_txt_save_path = '/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/bn_data.txt'
en_txt_save_path = '/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/en_data.txt'

In [7]:
# !wget 'https://raw.githubusercontent.com/sadhiin/Attention-Is-All-You-Need/refs/heads/main/data/bn_data.txt'
# !wget 'https://raw.githubusercontent.com/sadhiin/Attention-Is-All-You-Need/refs/heads/main/data/en_data.txt'

# bn_txt_save_path = 'bn_data.txt'
# en_txt_save_path = 'en_data.txt'

In [8]:
with open(en_txt_save_path, 'r') as file:
    english_sentences = file.readlines()

with open(bn_txt_save_path, 'r') as file:
    bangla_sentences = file.readlines()

# # Limit Number of sentences
# TOTAL_SENTENCES = 200000
# english_sentences = english_sentences[:TOTAL_SENTENCES]
# bangla_sentences = bangla_sentences[:TOTAL_SENTENCES]

english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
bangla_sentences = [sentence.rstrip('\n') for sentence in bangla_sentences]

In [9]:
english_sentences[:3]

["janet’s ducks lay 16 eggs per day. she eats three for breakfast every morning and bakes muffins for her friends every day with four. she sells the remainder at the farmers' market daily for $2 per fresh duck egg. how much in dollars does she make every day at the farmers' market?",
 'a robe takes 2 bolts of blue fiber and half that much white fiber.  how many bolts in total does it take?',
 'josh decides to try flipping a house.  he buys a house for $80,000 and then puts in $50,000 in repairs.  this increased the value of the house by 150%.  how much profit did he make?']

In [10]:
bangla_sentences[:3]

['জেনেটের হাঁসগুলি প্রতিদিন 16টি করে ডিম পাড়ে। তিনি প্রতিদিন প্রাতরাশে তিনটি করে ডিম খান এবং বন্ধুদের জন্য প্রতিদিন চারটি ডিম দিয়ে মাফিন তৈরি করেন। অবশিষ্ট হাঁসের ডিমগুলি তিনি প্রতিদিন কৃষকদের বাজারে প্রতি ডিম $2 দরে বিক্রয় করেন। তিনি কৃষকদের বাজারে প্রতিদিন কত ডলার উপার্জন করেন?',
 'একটি পোশাক তৈরি করতে 2 রিল নীল সুতো এবং তার অর্ধেক সাদা সুতো লাগে। এটি তৈরি করতে মোট কতগুলি রিল লাগবে?',
 'জোশ একটি বাড়ি ফ্লিপ করার সিদ্ধান্ত নিয়েছেন। তিনি $80,000-এ একটি বাড়ি ক্রয় করলেন এবং $50,000 দিয়ে মেরামত করলেন। এর ফলে বাড়িটির মূল্য 150% বৃদ্ধি পেল। তিনি কত ডলার লাভ করলেন?']

In [11]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length bangla: {np.percentile([len(x) for x in bangla_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length bangla: 477.06
97th percentile length English: 468.18


In [12]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(bangla_sentences)):
    bangla_sentence, english_sentence = bangla_sentences[index], english_sentences[index]
    if is_valid_length(bangla_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(bangla_sentence, bangla_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(bangla_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 250
Number of valid sentences: 4


In [13]:
bangla_sentences = [bangla_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [14]:
bangla_sentences[:3]

['গ্রেচেনের কাছে 110টি মুদ্রা আছে। রৌপ্য মুদ্রার থেকে 30টি বেশি স্বর্ণ মুদ্রা আছে। গ্রেচেনের কাছে কতগুলি স্বর্ণ মুদ্রা আছে?',
 'পাঞ্চো একদিনে 20 মাইল হাঁটেন। সপ্তাহান্তে ব্যতিরেকে তিনি 10 মাইল হাঁটেন। এক সপ্তাহে তিনি কত মাইল হাঁটেন?',
 'জন 3 ঘণ্টা ধরে পিচ সংগ্রহ করেন। তিনি এক মিনিটে 2টি পিচ সংগ্রহ করতে পারেন। তিনি কতগুলি পিচ সংগ্রহ করেন?']

In [15]:
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

d_model = 512
batch_size = 2
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
bn_vocab_size = len(bangla_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          bn_vocab_size,
                          english_to_index,
                          bangla_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [16]:
# transformer

In [17]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, bangla_sentences):
        self.english_sentences = english_sentences
        self.bangla_sentences = bangla_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.bangla_sentences[idx]

In [18]:
dataset = TextDataset(english_sentences, bangla_sentences)

In [19]:
len(dataset)

4

In [20]:
dataset[1]

('pancho walks 20 miles a day. except on weekends when he walks 10 miles. how many miles does he walk in a week?',
 'পাঞ্চো একদিনে 20 মাইল হাঁটেন। সপ্তাহান্তে ব্যতিরেকে তিনি 10 মাইল হাঁটেন। এক সপ্তাহে তিনি কত মাইল হাঁটেন?')

In [21]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [22]:
from rich import print
# for batch_num, batch in enumerate(iterator):
#     print(batch)
#     if batch_num > 3:
#         break

[
    (
        'gretchen has 110 coins. there are 30 more gold coins than silver coins. how many gold coins does gretchen 
have?',
        'pancho walks 20 miles a day. except on weekends when he walks 10 miles. how many miles does he walk in a 
week?'
    ),
    (
        'গ্রেচেনের কাছে 110টি মুদ্রা আছে। রৌপ্য মুদ্রার থেকে 30টি বেশি স্বর্ণ মুদ্রা আছে। গ্রেচেনের কাছে কতগুলি স্বর্ণ মুদ্রা আছে?',
        'পাঞ্চো একদিনে 20 মাইল হাঁটেন। সপ্তাহান্তে ব্যতিরেকে তিনি 10 মাইল হাঁটেন। এক সপ্তাহে তিনি কত মাইল হাঁটেন?'
    )
]

[
    (
        'john collects peaches for 3 hours.  he can collect 2 peaches a minute.  how many peaches does he 
collect?',
        'james has 6 more candies than robert. john has twice as many candies as robert. if john has 54 candies, 
how many more candies does john have than james?'
    ),
    (
        'জন 3 ঘণ্টা ধরে পিচ সংগ্রহ করেন। তিনি এক মিনিটে 2টি পিচ সংগ্রহ করতে পারেন। তিনি কতগুলি পিচ সংগ্রহ করেন?',
        'জেমসের কাছে রবার্টের থেকে 6টি ক্যান্ডি বেশি আছে। জনের কাছে রবার্টের থেকে দুইগুণ বেশি ক্যান্ডি আছে। জনের কাছে যদি 54টি ক্যান্ডি থাকে তাহলে জনের কাছে জেমসের থেকে
কতগুলি ক্যান্ডি বেশি আছে?'
    )
]

In [23]:
from torch import nn
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

criterian = nn.CrossEntropyLoss(ignore_index=bangla_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
NEG_INFTY = -1e9

def create_masks(eng_batch, bn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, bn_sentence_length = len(eng_batch[idx]), len(bn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      bn_chars_to_padding_mask = np.arange(bn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, bn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, bn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, bn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

Modify mask such that the padding tokens cannot look ahead.
In Encoder, tokens before it should be -1e9 while tokens after it should be -inf.


Note the target mask starts with 2 rows of non masked items: https://github.com/SamLynnEvans/Transformer/blob/master/Beam.py#L55


In [25]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

num_gpus = torch.cuda.device_count()
print(f"Number of GPUs: {num_gpus}")


gpu_index=0
device = torch.device(f'cuda:{gpu_index}')  # e.g., 'cuda:0', 'cuda:1'
# Check memory allocation
if torch.cuda.is_available():
    print(f"Allocated memory: {torch.cuda.memory_allocated()} bytes")
    print(f"Cached memory: {torch.cuda.memory_cached()} bytes")



transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        torch.cuda.empty_cache()
        
        transformer.train()
        eng_batch, bn_batch = batch

        # assert torch.all(torch.isfinite(eng_batch)), "English batch Input contains NaN or Inf values!"
        # assert torch.all(torch.isfinite(bn_batch)), "Bangla batch Input contains NaN or Inf values!"
        del batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, bn_batch)
        
        optim.zero_grad()
        bn_predictions = transformer(eng_batch,
                                     bn_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(bn_batch, start_token=False, end_token=True)
        loss = criterian(
            bn_predictions.view(-1, bn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == bangla_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"bangla Translation: {bn_batch[0]}")
            bn_sentence_predicted = torch.argmax(bn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in bn_sentence_predicted:
              if idx == bangla_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_bangla[idx.item()]
            print(f"bangla Prediction: {predicted_sentence}")


            transformer.eval()
            bn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, bn_sentence)
                predictions = transformer(eng_sentence,
                                          bn_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_bangla[next_token_index]
                bn_sentence = (bn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {bn_sentence}")
            print("-------------------------------------------")

CUDA is available!

Current device: 0

Device name: Tesla T4

Number of GPUs: 1

Allocated memory: 0 bytes

/tmp/ipykernel_30957/1467383845.py:17: FutureWarning: `torch.cuda.memory_cached` has been renamed to `torch.cuda.memory_reserved`
  print(f"Cached memory: {torch.cuda.memory_cached()} bytes")


Cached memory: 0 bytes

Epoch 0

../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [169,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [169,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [169,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [169,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [169,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [169,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [169,0,0], 

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Inference

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  bn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, bn_sentence)
    predictions = transformer(eng_sentence,
                              bn_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_bangla[next_token_index]
    bn_sentence = (bn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return bn_sentence[0]

In [ ]:
translation = translate("what should we do when the day starts?")
print(translation)

In [ ]:
translation = translate("how is this the truth?")
print(translation)


In [ ]:
translation = translate("the world is a large place with different people")
print(translation)

In [ ]:
translation = translate("my name is ajay")
print(translation)


ನಾನು ಕುಟುಂಬದ ಹೆಸರು<END>


In [ ]:
translation = translate("i cannot stand this smell")
print(translation)

In [ ]:
translation = translate("noodles are the best")
print(translation)

ಯಾವುದೇ ಕಾರಣಗಳು ಇಲ್ಲ<END>


In [ ]:
translation = translate("why care about this?")
print(translation)

ಏಕೆ ಕಾರಣ ಏನು?<END>


This translated pretty well : "What is the reason. Why" without punctuation.

In [ ]:
translation = translate("this is the best thing ever")
print(translation)

The translation : "This is very unusual"

In [ ]:
translation = translate("i am here")
print(translation)


Translation: "I have heard".
This is why word based translator may perform better than character translator. This is actually very good at optimizing the objective of the current transformer even though the translation is off.

In [ ]:
translation = translate("click this")
print(translation)

In [ ]:
translation = translate("where is the mall?")
print(translation)

In [ ]:
translation = translate("what should we do?")
print(translation)

This is correct; but it absolutely fumbles on the next one

In [ ]:
translation = translate("today, what should we do")
print(translation)

In [ ]:
translation = translate("why did they activate?")
print(translation)
# ಅವರು ಏಕೆ ಸಕ್ರಿಯಗೊಳಿಸಿದರು?

In [ ]:
translation = translate("why did they do this?")
print(translation)
# ಅವರು ಇದನ್ನು ಏಕೆ ಮಾಡಿದರು?

That turned out well!

In [ ]:
translation = translate("i am well.")
print(translation)
# ನಾನು ಆರಾಮವಾಗಿದ್ದೇನೆ

ನಾನು ನಾನು ಕೊಡುತ್ತೇನೆ.<END>


Translation: "I will give you something"

In [ ]:
translation = translate("whats the word on the street?")
print(translation)

ಇದರ ಬಗ್ಗೆ ಏನು?<END>


Kind of close semantically. Translation is something like: "What is this about"

## Insights

- When training, we can treat every alphabet as a single unit instead of splitting it into it's corresponding parts to preserve meaning. For example, ಮಾ should be 1 unit when comuting a loss. It should not be decomposed into ಮ + ఆ
- Using word-based or BPE based tokenizations may help mitigate (1). Also, we will get valid word (or BPE) units if we do so.
- Make sure the training set has a large variety of sentences that are not just about one topic like "work" and "government"
- Increase the number of encoder / decoder units for better translations. It was set to the minimum of 1 of each unit here.
- Create a translator with a language you understand ideally.

Overall, this model definately learned something. And you can use other languages instead of this kannada language and might see better luck